In [1]:
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from skimage.metrics import mean_squared_error, structural_similarity, peak_signal_noise_ratio

import torch
import torch.nn as nn
import torch.optim as optim

from utils import check_torch, pix2vid_dataset, NeuralPix2Vid

In [2]:
NR, NT = 1272, 60
NX, NY = 64, 64
milli  = 1e-3
mega   = 1e6
Darcy  = 9.869233e-13
psi2pa = 6894.75729
co2rho = 686.5400
sec2yr = 1/(3600*24*365.25)
device = check_torch()

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.3.1.post300 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3090
Torch device: cuda
------------------------------------------------------------


In [3]:
# x_inputs = Xt[:4].to(device)
# c_inputs = ct[:4].to(device)
x_inputs = torch.rand(2, 4, 8, 8).to(device)
c_inputs = torch.rand(2, 30, 5).to(device)
print(x_inputs.shape, c_inputs.shape)
print('-'*20)

model = NeuralPix2Vid(device=device).to(device)
out1, out2 = model(x_inputs, c_inputs)
print(out1.shape, out2.shape)

torch.Size([2, 4, 8, 8]) torch.Size([2, 30, 5])
--------------------
torch.Size([2, 30, 2, 8, 8]) torch.Size([2, 30, 1, 8, 8])


In [ ]:
fig,

In [ ]:
timesteps = np.load('timesteps.npy')
deltaTime = np.concatenate([np.ones((30)), np.ones((30))*50])
print('timesteps: {} | deltaT: {}'.format(len(timesteps), np.unique(deltaTime)))
(Xt, ct, y1t, y2t, all_volumes, idx), (trainloader, validloader) = pix2vid_dataset()

In [ ]:
model = NeuralPix2Vid().to(device)
print('# parameters: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

criterion1 = nn.MSELoss().to(device)
criterion2 = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs, monitor = 10, 1
train_loss, valid_loss = [], []
for epoch in range(epochs):
    epoch_train_loss = []
    model.train()
    for i, (x,y) in enumerate(trainloader):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        u = model(x)
        loss = criterion(y, u)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))
    # validation
    model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        for i, (x,y) in enumerate(validloader):
            x, y = x.to(device), y.to(device)
            u = model(x)
            loss = criterion(y, u)
            epoch_valid_loss.append(loss.item())
    valid_loss.append(np.mean(epoch_valid_loss))
    # progress
    if (epoch+1) % monitor == 0:
        print('Epoch: [{}/{}] | Train Loss: {:.5f} | Valid Loss: {:.5f}'.format(epoch+1, epochs, train_loss[-1], valid_loss[-1]))

torch.save(model.state_dict(), 'model.pth')
losses = pd.DataFrame({'train': train_loss, 'valid': valid_loss})
losses.to_csv('losses.csv', index=False)

In [ ]:
y1_pred = model(X_train[:20].to(device)).detach().cpu().numpy()
print(y1_pred.shape)

In [ ]:
sample = 10
fig, axs = plt.subplots(2, 10, figsize=(20,4), sharex=True, sharey=True)
for i in range(2):
    for j in range(10):
        ax = axs[i,j]
        k = j*3
        ax.imshow(y1_train[sample, i, :, :, k], cmap='turbo')
plt.tight_layout()
plt.show()